In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
X, y = make_regression()

# 1. Designing Clean Code with Reusable Structure

### Imperative

In [3]:
# no structure, hard to reuse/maintain
model = LinearRegression()

model.fit(X, y)

preds = model.predict(X)

What if I want to:
* add some print statements
* store predictions as an attribute
* add option to plot in predict method
* add option to save the model in predict method
* ...

### OOP

In [30]:
# easier to maintain/extend and read
class LinearModel:
    def __init__(self):
        self.model = LinearRegression()

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

In [31]:
model = LinearModel()

model.fit(X, y)

preds = model.predict(X)

#### Simple NN

In [ ]:
class SimpleNNModel:
    def __init__(self, input_dim, hidden_dim=16, lr=1e-3):
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        self.loss_fn = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

    def fit(self, X, y, epochs=100, verbose=True):
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

        for epoch in range(epochs):
            self.model.train()
            preds = self.model(X)
            loss = self.loss_fn(preds, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            if verbose and epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

    def predict(self, X):
        self.model.eval()
        X = torch.tensor(X, dtype=torch.float32)
        with torch.no_grad():
            preds = self.model(X).numpy().flatten()
        self.predictions = preds
        return preds

##### What if someone:

* does not follow our structure and uses a different naming convention?

![image](images/derp.png)

### OOP with ABC

In [3]:
from abc import ABC, abstractmethod

class BaseModel(ABC):

    @abstractmethod
    def fit(self, X, y):
        """Train the model using input features X and target y."""
        pass

    @abstractmethod
    def predict(self, X):
        """Generate predictions for input features X."""
        pass

    @abstractmethod
    def score(self, X, y):
        """Optional: Return a default scoring metric (e.g., MSE)."""
        preds = self.predict(X)
        return ((preds - y) ** 2).mean()



# 2. Readable, Safe, and Maintainable Code

What if someone:
- passes the wrong type?
- uses the neural network assuming the input should be a tensor?

### Beartype

In [4]:
from beartype import beartype

In [ ]:
## Show live solution

# ...

In [20]:
model = SimpleNNModel(input_dim=X.shape[1], hidden_dim=16, lr=1e-3)

model.fit(X, y)

preds = model.predict(X)

Epoch 0, Loss: 13397.9111
Epoch 10, Loss: 13365.1367
Epoch 20, Loss: 13332.0967
Epoch 30, Loss: 13295.9834
Epoch 40, Loss: 13254.6553
Epoch 50, Loss: 13206.3633
Epoch 60, Loss: 13149.3584
Epoch 70, Loss: 13082.6914
Epoch 80, Loss: 13003.7676
Epoch 90, Loss: 12912.2354


### Final Version

In [ ]:
@beartype
class SimpleNNModel:
    """A simple feedforward neural network model for regression tasks.

    Args:
        input_dim (int): Number of input features.
        hidden_dim (int, optional): Number of hidden units in the first layer. Default is 16.
        lr (float, optional): Learning rate for the optimizer. Default is 1e-3.
    """

    def __init__(self, input_dim: int, hidden_dim: int | None = 16, lr: float | None = 1e-3):
        """Initialize the model, loss function, and optimizer."""
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        self.loss_fn = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

    def fit(self, X: np.ndarray, y: np.ndarray, epochs: int | None = 100, verbose: bool | None = True) -> None:
        """Train the model using input features X and target y.

        Args:
            X (np.ndarray): Input features.
            y (np.ndarray): Target variable.
            epochs (int, optional): Number of training epochs. Default is 100.
            verbose (bool, optional): If True, print loss every 10 epochs. Default is True.
        Returns:
            None
        """
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

        for epoch in range(epochs):
            self.model.train()
            preds = self.model(X)
            loss = self.loss_fn(preds, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            if verbose and epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Generate predictions for input features X.
        
        Args:
            X (np.ndarray): Input features.
        Returns:
            np.ndarray: Predicted values.
        """
        self.model.eval()
        X = torch.tensor(X, dtype=torch.float32)
        with torch.no_grad():
            preds = self.model(X).numpy().flatten()
        self.predictions = preds
        return preds